# PART 1: SOURCING AND CLEANING DATA

In this notebook, I'll be creating 3 dataframes to analyze 311 complaints in NYC in 2017. The databases are the following:

1. NYC 311 Service Request Data
2. 2010 Census Population Data
3. Zip codes for NYC boroughs

In [66]:
import os
import sys
import time
import requests
import csv

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

from io import StringIO

In [2]:
def timeElapsed(start):
    elapsed = time.time() - start
    print(time.strftime("%M:%S", time.gmtime(elapsed)))

## NYC 311 Service Request Data
First, I'll take a few probes at the NYC 311 database to see what I'm working with. Much of this has been removed for readability.

In [68]:
# Get application token from local file, if available

try:
    with open('api_nycOpenData.csv') as f:
        reader = csv.reader(f)
        api_vals = next(reader)
    HEADERS = {api_vals[0] : api_vals[1]}
    
except:
    print('File not found')
    HEADERS = {}
    
print(HEADERS)

{'X-App-Token': 'oKcGXju2HqwSELuW5I9xfRkC0'}


In [69]:
pd.set_option('display.max_columns', 60)

# Start off with a simple call of 2019 complaints to get a feel for the dataset
apiURL = "https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?$where=date_extract_y(created_date)=2019"

resp = requests.get(apiURL, headers=HEADERS)
csvIO = StringIO(resp.text)
resp_df = pd.read_csv(csvIO)
resp_df

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,community_board,complaint_type,created_date,cross_street_1,cross_street_2,descriptor,due_date,facility_type,incident_address,incident_zip,intersection_street_1,intersection_street_2,landmark,latitude,location,location_address,location_city,location_state,location_type,location_zip,longitude,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,vehicle_type,x_coordinate_state_plane,y_coordinate_state_plane
0,NaN,DCAS,DCAS,NaN,Unspecified,NaN,NaN,1-1-1671254420,NaN,NaN,0 Unspecified,Question,2019-01-17T13:32:28.000,NaN,NaN,Administration,2019-01-31T13:32:55.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,Your comments have been submitted to the Depar...,NaN,Email Sent,NaN,NaN,NaN,41446020,NaN,NaN,NaN
1,ADDRESS,DEP,Department of Environmental Protection,3.060850e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-02-01T10:25:00.000,10 BROOKLYN,Water System,2019-01-17T10:42:00.000,RIDGE BLVD,3 AVE,Leak (Use Comments) (WA2),NaN,NaN,257 92 STREET,11209.0,NaN,NaN,NaN,40.619616,POINT (-74.033957080702 40.619615968635),NaN,NaN,NaN,NaN,NaN,-74.033957,PHONE,BROOKLYN,Unspecified,2019-02-01T10:25:00.000,The Department of Environmental Protection has...,NaN,Closed,92 STREET,NaN,NaN,41445885,NaN,974823.0,165021.0
2,ADDRESS,DOF,Condo or CoOp Unit,1.011668e+09,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-01-19T10:37:55.000,07 MANHATTAN,DOF Property - Reduction Issue,2019-01-17T14:11:15.000,NaN,NaN,Condo or Co-op Abatement,2019-01-24T14:11:15.000,NaN,NaN,10023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Condo Unit,NaN,NaN,PHONE,MANHATTAN,Unspecified,2019-01-19T10:37:55.000,See notes.,NaN,Closed,NaN,NaN,NaN,41445970,NaN,NaN,NaN
3,ADDRESS,HPD,Department of Housing Preservation and Develop...,3.012610e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-01-17T02:07:38.000,08 BROOKLYN,HEAT/HOT WATER,2019-01-14T21:44:41.000,NaN,NaN,ENTIRE BUILDING,NaN,NaN,511 EASTERN PARKWAY,11216.0,NaN,NaN,NaN,40.669917,POINT (-73.951342064433 40.669917102811),NaN,NaN,NaN,RESIDENTIAL BUILDING,NaN,-73.951342,MOBILE,BROOKLYN,Unspecified,2019-01-17T02:07:38.000,The Department of Housing Preservation and Dev...,NaN,Closed,EASTERN PARKWAY,NaN,NaN,41445576,NaN,997748.0,183349.0
4,ADDRESS,DSNY,BCC - Brooklyn South,3.068290e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-01-16T15:15:00.000,15 BROOKLYN,Derelict Vehicles,2019-01-16T15:15:00.000,AVENUE R,AVENUE S,14 Derelict Vehicles,NaN,DSNY Garage,1857 EAST 23 STREET,11229.0,NaN,NaN,NaN,40.605389,POINT (-73.949584821318 40.605389149211),NaN,NaN,NaN,Street,NaN,-73.949585,ONLINE,BROOKLYN,Unspecified,2019-01-16T15:15:00.000,The Department of Sanitation removed the items.,NaN,Pending,EAST 23 STREET,NaN,NaN,41445981,NaN,998249.0,159840.0
5,ADDRESS,DSNY,BCC - Brooklyn North,3.033400e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-01-17T12:00:00.000,04 BROOKLYN,Derelict Vehicles,2019-01-17T09:27:00.000,BUSHWICK AVENUE,EVERGREEN AVENUE,14 Derelict Vehicles,NaN,DSNY Garage,67 PALMETTO STREET,11221.0,NaN,NaN,NaN,40.691006,POINT (-73.919424688086 40.691006489545),NaN,NaN,NaN,Street,NaN,-73.919425,PHONE,BROOKLYN,Unspecified,2019-01-17T09:27:00.000,The Department of Sanitation removed the items.,NaN,Pending,PALMETTO STREET,NaN,NaN,41445913,NaN,1006595.0,191039.0
6,ADDRESS,DSNY,Bronx 04,2.025260e+09,BRONX,NaN,NaN,NaN,BRONX,2019-01-17T12:00:00.000,04 BRONX,Sanitation Condition,2019-01-16T10:07:00.000,UNNAMED STREET,WEST 166 STREET,15 Street Cond/Dump-Out/Drop-Off,NaN,NaN,1100 UNIVERSITY AVENUE,10452.0,NaN,NaN,NaN,40.835894,POINT (-73.929436573784 40.835894295045),NaN,NaN,NaN,Street,NaN,-73.929437,PHONE,BRONX,Unspecified,2019-01-17T12:00:00.000,The Department of Sanitation removed the items.,NaN,Closed,UNIVERSITY AVENUE,NaN,NaN,41445058,NaN,1003776.0,243824.0
7,ADDRESS,DSNY,Department of Sanit

In [70]:
apiURL = "https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?" \
          "$where=date_extract_y(created_date)=2017" \
          "&$select=count(created_date)"
resp = requests.get(apiURL, headers=HEADERS)
totalRecords = int(resp.text.split('"')[-2])

# Look through column fields to see which data may/not be relevant, get indices.
for idx, col in enumerate(resp_df.columns):
    print(idx, col)

0 address_type
1 agency
2 agency_name
3 bbl
4 borough
5 bridge_highway_direction
6 bridge_highway_name
7 bridge_highway_segment
8 city
9 closed_date
10 community_board
11 complaint_type
12 created_date
13 cross_street_1
14 cross_street_2
15 descriptor
16 due_date
17 facility_type
18 incident_address
19 incident_zip
20 intersection_street_1
21 intersection_street_2
22 landmark
23 latitude
24 location
25 location_address
26 location_city
27 location_state
28 location_type
29 location_zip
30 longitude
31 open_data_channel_type
32 park_borough
33 park_facility_name
34 resolution_action_updated_date
35 resolution_description
36 road_ramp
37 status
38 street_name
39 taxi_company_borough
40 taxi_pick_up_location
41 unique_key
42 vehicle_type
43 x_coordinate_state_plane
44 y_coordinate_state_plane


### Sources

There are 2 options for obtaining this NYC 311 data:
1. Download the full dataset (CSV file)
2. Use the Socrata API to query for the data

The full dataset is prohibitively large for my working environment, so I'm choosing the API route. I'm limiting each request to 50,000 records (the max value for v2.0 of this API. I *believe* this endpoint is 2.1 and has no such limit, but I'll keep it to be safe), and will be paging through the data. A count query shows that there are ~2.4M records for 2017, so I'll need to be a little structured about how I make the query.

Most requests finish in a few seconds, but the server seem to does slow down (~30s per request) once I reach about 2M records. This typically finishes in 5 minutes. I currently only kill the process on an unsuccessul response code, but more handling should be implemented for other codes and timeouts. The complete dataframe will get stored to a pickle file to avoid making these API requests for repeated runs.

I initially downloaded the full records so I could have more data to cross-reference for cleaning purposes (i.e. fill in missing zip code and borough fields). Currently when I convert to a dataframe, I throw most of this data away, but this could be used if a more thorough cleaning were undertaken.

Most columns have data of mixed type, so I opted to let Pandas assign a dtype of object/string. I do set the dtype as a string for the zip code since I don't need to do any numeric processing on it and several entries have non-numeric characters.

In [7]:
def buildNYCData(numRows):
    df_list = list()
    
    numOffset = 0
    count = numRows

    start = time.time()
    apiURL = "https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?" \
             "$where=date_extract_y(created_date)=2017" \
             "&$select=count(created_date)"
    resp = requests.get(apiURL, headers=HEADERS)
    timeElapsed(start)
    print(resp.text.split('"')[-2], 'total records\n')

    while count == numRows:
        apiURL = "https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?" \
                 "$where=date_extract_y(created_date)=2017" \
                 "&$limit=" + str(numRows) + "&$offset=" + str(numOffset)
        
        start = time.time()
        resp = requests.get(apiURL, headers=HEADERS)
        timeElapsed(start)

        print(str(resp.status_code), ': records =', numOffset+numRows)
        if resp.status_code != 200:
            print('API request failed')
            break

        start = time.time()
        csvIO = StringIO(resp.text)
        df_list.append(pd.read_csv(csvIO, dtype={'incident_zip':str},
                       usecols=['borough', 'city', 'complaint_type', 'created_date', 'incident_zip']) \
                               [['created_date', 'complaint_type', 'incident_zip', 'borough', 'city']])
                       #, low_memory=False))# Keeping the warnings

        timeElapsed(start)
        print('\n')
        
        count = df_list[-1].shape[0]
        numOffset += count
        
    return pd.concat(df_list, ignore_index=True)

In [10]:
numRows = 50000

# Don't query the data for repeated runs
if os.path.isfile('nyc1_df.pkl'):
    nyc1_df = pd.read_pickle('nyc1_df.pkl')
else:
    nyc1_df = buildNYCData(numRows)

00:00
2461208 total records

00:01
200 : records = 50000
00:00


00:01
200 : records = 100000
00:00


00:02
200 : records = 150000
00:00


00:02
200 : records = 200000
00:00


00:01
200 : records = 250000
00:00


00:06
200 : records = 300000
00:00


00:02
200 : records = 350000
00:00


00:02
200 : records = 400000
00:00


00:04
200 : records = 450000
00:00


00:08
200 : records = 500000
00:00


00:02
200 : records = 550000


/Users/master/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


00:00


00:01
200 : records = 600000
00:00


00:02
200 : records = 650000
00:00


00:01
200 : records = 700000
00:00


00:01
200 : records = 750000
00:00


00:01
200 : records = 800000
00:00


00:01
200 : records = 850000
00:00


00:01
200 : records = 900000
00:00


00:02
200 : records = 950000
00:00


00:02
200 : records = 1000000
00:00


00:02
200 : records = 1050000
00:00


00:01
200 : records = 1100000
00:00


00:02
200 : records = 1150000
00:00


00:02
200 : records = 1200000
00:00


00:02
200 : records = 1250000
00:00


00:01
200 : records = 1300000
00:00


00:01
200 : records = 1350000
00:00


00:02
200 : records = 1400000
00:00


00:02
200 : records = 1450000
00:00


00:02
200 : records = 1500000
00:00


00:02
200 : records = 1550000
00:00


00:02
200 : records = 1600000
00:00


00:01
200 : records = 1650000
00:00


00:02
200 : records = 1700000
00:00


00:03
200 : records = 1750000
00:00


00:01
200 : records = 1800000
00:00


00:01
200 : records = 1850000
00:00


00:02
200 : 

In [11]:
# Store in pickle file locally before cleanup
nyc1_df.to_pickle('nyc1_df.pkl')

 Now to inspect data and look for missing fields...
    
### complaint_type

As seen in the count list below, there are what seems to be duplicates (e.g. `'UNSANITARY CONDITION'` and `'Sanitation condition'`); however, the 311 submission fields online have categories and *subcategories*, many of which seem to change or be user submitted. These records seem to only have the subcategory, and so without context, it's possible I could be mixing complaints from entirely separate categories if I tried any combining.

I will, however, convert these strings to a uniform case to catch any matching categories. This will catch some extra `'PLUMBING'` complaints for example, but it falls outside the top 10.

In [32]:
nyc1_df['complaint_type'].value_counts(dropna=False).head(20)

Noise - Residential                    229314
HEAT/HOT WATER                         210158
Illegal Parking                        147525
Blocked Driveway                       137547
Street Condition                        98622
UNSANITARY CONDITION                    81801
Street Light Condition                  81537
Noise - Street/Sidewalk                 72629
Water System                            62853
Noise                                   62191
PAINT/PLASTER                           59566
PLUMBING                                50781
Request Large Bulky Item Collection     49057
Noise - Commercial                      46997
Sanitation Condition                    40665
General Construction/Plumbing           39503
Missed Collection (All Materials)       37933
Dirty Conditions                        37446
Rodent                                  36958
DOOR/WINDOW                             36347
Name: complaint_type, dtype: int64

In [34]:
# Look at count list again after str.upper()
nyc1_df['complaint_type'] = nyc1_df['complaint_type'].str.upper()
nyc1_df['complaint_type'].value_counts(dropna=False).head(20)

NOISE - RESIDENTIAL                    229314
HEAT/HOT WATER                         210158
ILLEGAL PARKING                        147525
BLOCKED DRIVEWAY                       137547
STREET CONDITION                        98622
UNSANITARY CONDITION                    81801
STREET LIGHT CONDITION                  81537
NOISE - STREET/SIDEWALK                 72629
WATER SYSTEM                            62853
NOISE                                   62191
PAINT/PLASTER                           59566
PLUMBING                                56893
REQUEST LARGE BULKY ITEM COLLECTION     49057
NOISE - COMMERCIAL                      46997
SANITATION CONDITION                    40665
GENERAL CONSTRUCTION/PLUMBING           39503
MISSED COLLECTION (ALL MATERIALS)       37933
DIRTY CONDITIONS                        37446
RODENT                                  36958
DOOR/WINDOW                             36347
Name: complaint_type, dtype: int64

### borough

There are "unspecified" boroughs that could likely be inferred from **incident_zip**, and that will be done later on with the aid of the NYC borough/zip code database.

Otherwise, there are *many* fields that could be used to fill in the blank (city, cross streets, intersection streets, park borough, etc.). Without more context from NYC 311's info page, it's unclear how to best correlate this data.

In [14]:
nyc1_df['borough'].value_counts(dropna=False)

BROOKLYN         768828
QUEENS           593648
MANHATTAN        481812
BRONX            455109
STATEN ISLAND    129194
Unspecified       32617
Name: borough, dtype: int64

In [63]:
bor_missing = len(nyc1_df[nyc1_df['borough'] == 'Unspecified'])
print('{} of {} borough records are missing. This represents {:.3f}% of our data.'
      .format(bor_missing, totalRecords, float(100*bor_missing)/totalRecords))

zip_fix = len(nyc1_df[(nyc1_df['incident_zip'].notna()) & (nyc1_df['borough'] == 'Unspecified')])
print('{} of these {} missing borough entries could be inferred from an available zip code.'
      .format(zip_fix, bor_missing))
print('This could correct up to {:.3f}% of our missing borough data'
      .format(100*float(zip_fix)/bor_missing))

32617 of 2461208 borough records are missing. This represents 1.325% of our data.
3781 of these 32617 missing borough entries could be inferred from an available zip code.
This could correct up to 11.592% of our missing borough data


### incident_zip

There are many missing and irregular entries for this field. I'm not sure there's a clear answer to filling in this data.

Clean up will involve making sure all zip codes are numeric and 5 characters or less.

In [35]:
nyc1_df['incident_zip'].value_counts(dropna=False)

NaN           96625
11226         42438
11385         35893
10467         34065
11207         31436
10452         29896
10458         29797
10453         29571
10468         28952
11208         28358
10456         26918
10031         26785
10457         26752
11221         25900
11225         25803
11206         25595
11213         24633
11234         24109
10472         23875
11212         23765
11209         23722
11203         23699
11233         23441
11216         22945
10314         22281
11215         22076
11220         22008
11230         21893
10032         21641
10466         21544
              ...  
60604             1
10567             1
10510             1
08063             1
43226             1
91716-0500        1
30006             1
07002             1
33131             1
10159             1
10853             1
10401             1
14901             1
11771             1
11010             1
80235             1
07108             1
13903             1
01234             1


In [64]:
zip_missing = len(nyc1_df[nyc1_df['incident_zip'].isna()])
print('{} of {} zip code records are missing. This represents {:.3f}% of our data.'
      .format(zip_missing, totalRecords, float(100*zip_missing)/totalRecords))

96625 of 2461208 zip code records are missing. This represents 3.926% of our data.


## 2010 Census Population Data

This database is available at Splitwise blog and contains US zip codes and population from the 2010 Census.

In [42]:
censusURL = "https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv"
census_df = pd.read_csv(censusURL, dtype={'Zip Code ZCTA':str})
census_df.head(10)

,Zip Code ZCTA,2010 Census Population
0,01001,16769
1,01002,29049
2,01003,10372
3,01005,5079
4,01007,14649
5,01008,1263
6,01009,741
7,01010,3609
8,01011,1370
9,01012,661


In [44]:
# Some zip codes are listed twice. Checked all zips that start with '100', '10', '11'. No duplicates.
census_df[census_df['Zip Code ZCTA'].str.startswith('11')]['Zip Code ZCTA'].value_counts();

In [45]:
# Write to file
census_df.to_pickle('census_df.pkl')

## Zip codes for NYC boroughs

This dataset will be needed to match zip codes from the Census dataset with its NYC borough. This is being read from an html table from an NYC enthusiast site. The online formatting will require some transformation here.

In [57]:
zipsURL = "https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php"
zips_df = pd.read_html(zipsURL, match='10001')[0]
print('{} rows x {} columns'.format(zips_df.shape[0], zips_df.shape[1]))
zips_df.head(20)

240 rows x 5 columns


,0,1,2,3,4
0,10001,Manhattan,NaN,10451,Bronx
1,10002,Manhattan,NaN,10452,Bronx
2,10003,Manhattan,NaN,10453,Bronx
3,10004,Manhattan,NaN,10454,Bronx
4,10005,Manhattan,NaN,10455,Bronx
5,10006,Manhattan,NaN,10456,Bronx
6,10007,Manhattan,NaN,10457,Bronx
7,10009,Manhattan,NaN,10458,Bronx
8,10010,Manhattan,NaN,10459,Bronx
9,10011,Manhattan,NaN,10460,Bronx


In [58]:
# The table is displayed twice in different order. Only need one side.
zips_df.drop(labels=[2,3,4], axis='columns', inplace=True)
print('{} rows x {} columns'.format(zips_df.shape[0], zips_df.shape[1]))
zips_df.head(20)

240 rows x 2 columns


,0,1
0,10001,Manhattan
1,10002,Manhattan
2,10003,Manhattan
3,10004,Manhattan
4,10005,Manhattan
5,10006,Manhattan
6,10007,Manhattan
7,10009,Manhattan
8,10010,Manhattan
9,10011,Manhattan


In [59]:
# Rename columns and check borough values
zips_df.set_axis(['zip_code', 'borough'], axis='columns', inplace=True)
zips_df['borough'].value_counts()

Manhattan    96
Queens       63
Brooklyn     43
Bronx        25
Staten       13
Name: borough, dtype: int64

In [60]:
# Will convert to uppercase and fix 'Staten' for consistency with previous datasets
zips_df['borough'] = zips_df['borough'].str.upper()
zips_df.loc[zips_df['borough'] == 'STATEN', 'borough'] = 'STATEN ISLAND'

# Convert zip code dtype to string
zips_df['zip_code'] = zips_df['zip_code'].astype(str)

In [61]:
#  Write to file
zips_df.to_pickle('zips_df.pkl')

In [ ]:
# Write cleaned up NYC dataframe to separate file
nyc1_df.to_pickle('nyc2_df.pkl')